In [1]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import Sequential,MaxPool2d,Linear,ReLU
from functions_pytorch import *
from pytorch_util import *
from pytorch_models import ConvBatchRelu
from albumentations_helper import create_transform
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
import pickle
import numpy as np
import pretrainedmodels
from functools import partial

In [2]:
## configs ##
color = True
#shapes = (3,224,224)
HalfBatch = 8
margin = -2
outDim = 200
distanceFun = l2_distance
distanceFun_np = l2_distance_np
TTASize = 4
pct = 0.1
multiple = 3

In [3]:
if color:
    with open('/home/will/Desktop/kaggle/Whale/train_df_color.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train_color.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df_color.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val_color.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)
else:
    with open('/home/will/Desktop/kaggle/Whale/train_df.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)

Set up dataloader

In [4]:
aug = Compose([RandomContrast(p=0.5),RandomBrightness(p=0.5),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=25,scale_limit=0.05,p=1),Cutout(p=0.5)])
transform = create_transform(aug)  

aug_test = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1)])
transform_test = create_transform(aug_test)    

In [5]:
gen_train = TripletGenerator(Ids_train,newWhale_train,transform)
gen_val = TripletGenerator(Ids_val,newWhale_val,transform_test)        
#gen_train = FunctionWrapOverDataset(gen_train,numpy2torch)
#gen_val = FunctionWrapOverDataset(gen_val,numpy2torch)
train_dl= DataLoader(gen_train,HalfBatch,True,num_workers=3,drop_last=True)
valid_dl = DataLoader(gen_val,HalfBatch,False,num_workers=3,drop_last=True)

Set up training

In [6]:
loc = Sequential(ConvBatchRelu(3,8,5,stride=2),\
                     ConvBatchRelu(8,16,5,stride=2),\
                     ConvBatchRelu(16,32,5,stride=2),\
                     ConvBatchRelu(32,64,5,stride=2))

In [7]:
fcNet = Sequential(Linear(7744,32),ReLU(True),Linear(32,6*multiple))

In [8]:
stn = SpatialTransformerNetMutiple(loc,fcNet,multiple,[3, 112, 112])

In [9]:
base = pretrainedmodels.resnet18()
base = fine_tune_pretrainedmodels(base, outDim)

In [10]:
model = STN_multiple_model(stn,base).to('cuda:0')

In [11]:
loss_func = loss_func_generator(HalfBatch,margin,distanceFun)

In [12]:
opt = Adam(trainable_parameter(model),lr=1e-4)
#opt = SGD(trainable_parameter(model),lr=3e-4)

In [13]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl)

epoch:0, train_loss:-0.2952669328123331, val_loss:-0.549985945224762
epoch:1, train_loss:-0.5167314027965069, val_loss:-0.5381356477737427
epoch:2, train_loss:-0.656345286154747, val_loss:-0.23699015378952026
epoch:3, train_loss:-0.49013974902629853, val_loss:-0.8242770433425903
epoch:4, train_loss:-0.7187960035681724, val_loss:-0.7648609280586243
Training completed in 212.52335715293884s


In [14]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl)

epoch:0, train_loss:-0.7536704321503639, val_loss:-1.0333362817764282
epoch:1, train_loss:-0.8645521814107895, val_loss:-0.9286816716194153
epoch:2, train_loss:-0.8533955804347992, val_loss:-0.8994772434234619
epoch:3, train_loss:-0.8535406195878983, val_loss:-1.1930184364318848
epoch:4, train_loss:-1.01813995013237, val_loss:-1.0756570100784302
Training completed in 212.13031125068665s


Fine-tune previous layers

In [ ]:
# for ResNet50
# set_requires_grad([model[1].layer4,model[1].layer3,model[1].layer2],True)
# opt = Adam([
#             {"params": model[1].layer4.parameters(), "lr": 1e-5},
#             {"params": model[1].layer3.parameters(), "lr": 1e-6},
#             {"params": model[1].layer2.parameters(), "lr": 5e-7},
#             {"params": model[1].last_linear.parameters(), "lr": 2e-5},
#             {"params": model[0].parameters(), "lr": 2e-5}
#             ])

In [25]:
set_requires_grad(model,True)
#name_list = ['denseblock3','transition3','denseblock4','norm5']
#set_requires_grad_paraList(gather_parameter_nameList(name_list,model),True)

In [26]:
opt = Adam([
            {"params": gather_parameter_nameList(['layer4'],model), "lr": 5e-5},
            {"params": gather_parameter_nameList(['layer3'],model), "lr": 2e-5},
            {"params": gather_parameter_nameList(['layer2'],model), "lr": 1e-5},
            {"params": gather_parameter_nameList(['layer1'],model), "lr": 5e-6},
            {"params": model.convNet.conv1.parameters(), "lr": 2e-6},
            {"params": model.convNet.bn1.parameters(), "lr": 2e-6},
            ],lr=8e-5)

In [27]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl)

epoch:0, train_loss:-1.1085381864786148, val_loss:-1.1494746208190918
epoch:1, train_loss:-1.2792052606582642, val_loss:-1.181570291519165
epoch:2, train_loss:-1.2663943541526794, val_loss:-1.2974693775177002
epoch:3, train_loss:-1.3377004085302353, val_loss:-1.3113305568695068
epoch:4, train_loss:-1.3569880262374878, val_loss:-1.3325304985046387
Training completed in 293.33615016937256s


In [28]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl)

epoch:0, train_loss:-1.3693908285617828, val_loss:-1.3452659845352173
epoch:1, train_loss:-1.407889396905899, val_loss:-1.2048245668411255
epoch:2, train_loss:-1.45262935962677, val_loss:-1.2281945943832397
epoch:3, train_loss:-1.4333325175285339, val_loss:-1.2810242176055908
epoch:4, train_loss:-1.4772603550434114, val_loss:-1.353670358657837
Training completed in 292.55577206611633s


Validation

In [17]:
pred_train_gen = PredictGenerator(Ids_train.Imgs.tolist(),transform_test,TTASize, True)
pred_val_gen = PredictGenerator(Ids_val.Imgs.tolist(),transform_test,TTASize, True)

In [18]:
feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

In [19]:
feature_train = feature_train.reshape(len(pred_train_gen),TTASize,outDim)
feature_val = feature_val.reshape(len(pred_val_gen),TTASize,outDim)

In [20]:
aggFun = partial(np.quantile,q=pct,axis=(1,2))

In [21]:
predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

In [22]:
mapping_dict = dict(zip(Ids_train.Id.values,Ids_train.index.values))
labels = Ids_val.Id.map(mapping_dict)

In [23]:
MAP(labels,predicts)

0.12482993197278912